# Introduction to loading data <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **Acknowledgement**: This notebook was originally created by [Digital Eath Australia (DEA)](https://www.ga.gov.au/about/projects/geographic/digital-earth-australia) and has been modified for use in the EY Data Science Program
* **Products used:** 
[ga_s2a_ard_nbar_granule](https://explorer.sandbox.dea.ga.gov.au/ga_s2a_ard_nbar_granule),
[ga_s2b_ard_nbar_granule](https://explorer.sandbox.dea.ga.gov.au/ga_s2b_ard_nbar_granule)
* **Prerequisites:** Users of this notebook should have a basic understanding of:
    * How to run a [Jupyter notebook](01_Jupyter_notebooks.ipynb)
    * The basic structure of the DEA [satellite datasets](02_DEA.ipynb)
    * Inspecting available [DEA products and measurements](03_Products_and_measurements.ipynb)
    

## Background
Loading data from the [Digital Earth Australia (DEA)](https://www.ga.gov.au/dea) instance of the [Open Data Cube](https://www.opendatacube.org/) requires the construction of a data query that specifies the what, where, and when of the data request.
Each query returns a [multi-dimensional xarray object](http://xarray.pydata.org/en/stable/) containing the contents of your query.
It is essential to understand the `xarray` data structures as they are fundamental to the structure of data loaded from the datacube.
Manipulations, transformations and visualisation of `xarray` objects provide datacube users with the ability to explore and analyse DEA datasets, as well as pose and answer scientific questions.

## Description
This notebook will introduce how to load data from the DEA datacube through the construction of a query and use of the `dc.load()` function.
Topics covered include:

1. Loading data using `dc.load()`
2. Interpreting the resulting `xarray.Dataset` object
    * Inspecting an individual `xarray.DataArray`
3. Customising parameters passed to the `dc.load()` function
    * Loading specific measurements
    * Loading data for coordinates in a custom coordinate reference system (CRS)
    * Projecting data to a new CRS and spatial resolution 
    * Specifying a specific spatial resampling method
4. Loading data using a reusable dictionary query
5. Loading matching data from multiple products using `like`
6. Adding a progress bar to the data load

***

## Getting started
To run this introduction to loading data from DEA, run all the cells in the notebook starting with the "Load packages" cell. For help with running notebook cells, refer back to the [Jupyter Notebooks notebook](01_Jupyter_notebooks.ipynb).

### Load packages
The `datacube` package is required to query the datacube database and load some data. 
The `with_ui_cbk` function from `odc.ui` enables a progress bar when loading large amounts of data.

In [1]:
import datacube
from odc.ui import with_ui_cbk

### Connect to the datacube
The next step is to connect to the datacube database.
The resulting `dc` datacube object can then be used to load data.
The `app` parameter is a unique name used to identify the notebook that does not have any effect on the analysis.

In [2]:
dc = datacube.Datacube(app="04_Loading_data")

## Loading data using `dc.load()`

Loading data from the datacube uses the [dc.load()](https://datacube-core.readthedocs.io/en/latest/dev/api/generate/datacube.Datacube.load.html) function.

The function requires the following minimum arguments:

* `product`: The data product to load (to revise DEA products, see the [Products and measurements](03_Products_and_measurements.ipynb) notebook).
* `x`: The spatial region in the *x* dimension.
* `y`: The spatial region in the *y* dimension. The dimensions ``longitude``/``latitude`` and ``x``/``y`` can be used interchangeably.
* `crs`: The geographical Cooridinate Reference System (CRS) that the *x* and *y* arguments are in.
* `time`: The temporal extent. The time dimension can be specified using a tuple of datetime objects or strings in the "YYYY", "YYYY-MM" or "YYYY-MM-DD" format. 
* `output_crs`: The CRS to return the data in.
* `resolution`: The resolution to return the data in. The units are set by the output_crs argument.

For example, to load the first week of January 2018 data from the [Sentinel-2A Analysis Ready Data NBAR product](https://explorer.sandbox.dea.ga.gov.au/ga_s2a_ard_nbar_granule) for Western Port Bay near Phillip Island in southern Victoria, use the following parameters:

* `product`: `ga_s2a_ard_nbar_granule`
* `x`: `(145.1, 145.2)`
* `y`: `(-38.4, -38.5)`
* `time`: `("2018-01-01", "2018-01-07")`
* `crs`: `"epsg:4326"`
* `output_crs`: `"epsg:4326"`
* `resolution`: `(-0.01, 0.01)`

Run the following cell to load all datasets from the `ga_s2a_ard_nbar_granule` product that match this spatial and temporal extent:

In [3]:
ds = dc.load(product="ga_s2a_ard_nbar_granule",
             x=(145.1, 145.2),
             y=(-38.4, -38.5),
             time=("2018-01-01", "2018-01-07"),
             crs="epsg:4326",
             output_crs="epsg:4326",
             resolution=(-0.01, 0.01))

print(ds)

<xarray.Dataset>
Dimensions:               (latitude: 10, longitude: 10, time: 1)
Coordinates:
  * time                  (time) datetime64[ns] 2018-01-03T00:11:01.026000
  * latitude              (latitude) float64 -38.41 -38.41 ... -38.48 -38.5
  * longitude             (longitude) float64 145.1 145.1 145.1 ... 145.2 145.2
    spatial_ref           int32 4326
Data variables:
    azimuthal_exiting     (time, latitude, longitude) float32 -150.63 ... -95.73842
    azimuthal_incident    (time, latitude, longitude) float32 -132.11162 ... -67.85103
    exiting               (time, latitude, longitude) float32 6.1649747 ... 8.596628
    incident              (time, latitude, longitude) float32 31.505491 ... 33.584988
    relative_azimuth      (time, latitude, longitude) float32 30.557167 ... 30.78624
    relative_slope        (time, latitude, longitude) float32 18.518387 ... 27.88739
    satellite_azimuth     (time, latitude, longitude) float32 101.02727 ... 101.03792
    satellite_view     

### Interpreting the resulting `xarray.Dataset`
The variable `ds` has returned an `xarray.Dataset` containing all data that matched the spatial and temporal query parameters inputted into `dc.load`.

*Dimensions* 

* This header identifies the number of timesteps returned in the search (`time: 1`) as well as the number of pixels in the `latitude` and `longitude` directions of the data query.

*Coordinates* 

* `time` identifies the date attributed to each returned timestep.
* `latitude` and `longitude` are the coordinates for each pixel within the spatial bounds of the query.

*Data variables*

* These are the measurements available for the nominated product. 
For every date (`time`) returned by the query, the measured value at each pixel (`latitude`, `longitude`) is returned as an array for each measurement.
Each data variable is itself an `xarray.DataArray` object ([see below](#Inspecting-an-individual-xarray.DataArray)). 

*Attributes*

* `crs` identifies the coordinate reference system (CRS) of the loaded data. 

### Inspecting an individual `xarray.DataArray`
The `xarray.Dataset` loaded above is itself a collection of individual `xarray.DataArray` objects that hold the actual data for each data variable/measurement. 
For example, all measurements listed under _Data variables_ above (e.g. `nbar_blue`, `nbar_green`, `nbar_red`, `nbar_nir_1`, `nbar_nir_2`, `nbar_swir_1`) are `xarray.DataArray` objects.

These `xarray.DataArray` objects can be inspected or interacted with by using either of the following syntaxes:
```
ds["measurement_name"]
```
or
```
ds.measurement_name
```

The ability to access individual variables means that these can be directly viewed, or further manipulated to create new variables. 
For example, run the following cell to access data from the near infra-red satellite band (i.e. `nbar_nir_1`):

In [4]:
print(ds.nbar_nir_1)

<xarray.DataArray 'nbar_nir_1' (time: 1, latitude: 10, longitude: 10)>
array([[[ 812, 1043,  981, 1052, 1018,  776,  488,  573,  596,  705],
        [ 777,  594,  727,  952,  678,  534,  458,  477,  552,  624],
        [1247,  545,  815,  900,  700,  474,  474,  395, 2404,  783],
        [ 592,  543,  775,  858,  656,  434,  421,  418,  636,  736],
        [ 503,  526,  831,  640, 1264,  358,  470,  453,  526,  595],
        [ 341,  480,  545,  386,  234,  358, 2769, 1560,  546,  474],
        [ 342,  351,  348,  349,  269,  240,  353,  534, 2242, 2759],
        [ 341,  358,  369,  289,  350,  247,  396, 2477, 3340, 1412],
        [ 263,  401,  476,  387,  226,  471, 2692, 2928,  804, 5028],
        [ 329,  446,  322,  185,  275,  316, 2782, 3125, 2681, 2811]]],
      dtype=int16)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-03T00:11:01.026000
  * latitude     (latitude) float64 -38.41 -38.41 -38.43 ... -38.48 -38.48 -38.5
  * longitude    (longitude) float64 145.1 145.1

Note that the object header informs us that it is an `xarray.DataArray` containing data for the `nir` satellite band. 

Like an `xarray.Dataset`, the array also includes information about the data's **dimensions** (i.e. `(time: 1, latitude: 10, longitude: 10)`), **coordinates** and **attributes**.
This particular data variable/measurement contains some additional information that is specific to the `nbar_nir_1` band, including details of array's nodata value (i.e. `nodata: -999`).

> For a more in-depth introduction to `xarray` data structures, refer to the [official xarray documentation](http://xarray.pydata.org/en/stable/data-structures.html)

## Customising the `dc.load()` function

The `dc.load()` function can be tailored to refine a query.

Customisation options include:

* `measurements:` This argument is used to provide a list of measurement names to load, as listed in `dc.list_measurements()`. 
For satellite datasets, measurements contain data for each individual satellite band (e.g. near infrared). 
If not provided, all measurements for the product will be returned.
* `crs:` The coordinate reference system (CRS) of the query's `x` and `y` coordinates is assumed to be `WGS84`/`EPSG:4326` unless the `crs` field is supplied, even if the stored data is in another projection or the `output_crs` is specified. 
The `crs` parameter is required if the query's coordinates are in any other CRS.
* `group_by:` Satellite datasets based around scenes can have multiple observations per day with slightly different time stamps as the satellite collects data along its path.
These observations can be combined by reducing the `time` dimension to the day level using `group_by=solar_day`.
* `output_crs` and `resolution`: To reproject or change the resolution the data, supply the `output_crs` and `resolution` fields.    
* `resampling`: This argument allows you to specify a custom spatial resampling method to use when data is reprojected into a different CRS. 

Example syntax on the use of these options follows in the cells below.

> For help or more customisation options, run `help(dc.load)` in an empty cell or visit the function's [documentation page](https://datacube-core.readthedocs.io/en/latest/dev/api/generate/datacube.Datacube.load.html)


### Specifying measurements
By default, `dc.load()` will load *all* measurements in a product.

To load data from the `red`, `green` and `blue` satellite bands only, add `measurements=["nbar_red", "nbar_green", "nbar_blue"]` to the query:

In [5]:
# Note the optional inclusion of the measurements list
ds_rgb = dc.load(product="ga_s2a_ard_nbar_granule",
                 measurements=["nbar_red", "nbar_green", "nbar_blue"],
                 x=(145.1, 145.2),
                 y=(-38.4, -38.5),
                 time=("2018-01-01", "2018-01-07"),
                 crs="epsg:4326",
                 output_crs="epsg:4326",
                 resolution=(-0.01, 0.01))

print(ds_rgb)

<xarray.Dataset>
Dimensions:      (latitude: 10, longitude: 10, time: 1)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-03T00:11:01.026000
  * latitude     (latitude) float64 -38.41 -38.41 -38.43 ... -38.48 -38.48 -38.5
  * longitude    (longitude) float64 145.1 145.1 145.1 ... 145.2 145.2 145.2
    spatial_ref  int32 4326
Data variables:
    nbar_red     (time, latitude, longitude) int16 714 889 788 ... 1065 1274
    nbar_green   (time, latitude, longitude) int16 878 1089 1033 ... 909 786 957
    nbar_blue    (time, latitude, longitude) int16 831 1070 1075 ... 641 573 728
Attributes:
    crs:           epsg:4326
    grid_mapping:  spatial_ref


Note that the **Data variables** component of the `xarray.Dataset` now includes only the measurements specified in the query (i.e. the `nbar_red`, `nbar_green` and `nbar_blue` satellite bands).

### Loading data for coordinates in any CRS
By default, `dc.load()` assumes that the queried `x` and `y` coordinates are in the `WGS84`/`EPSG:4326` CRS.
If these coordinates are in a different coordinate system, specify this using the `crs` parameter.

The example cell below loads data for a set of `x` and `y` coordinates defined in GDA94 / MGA zone 55 (`EPSG:28355`), ensuring that the `dc.load()` function accounts for this by including `crs="EPSG:28355"`:

In [6]:
# Note the new `x` and `y` coordinates and `crs` parameter
ds_custom_crs = dc.load(product="ga_s2a_ard_nbar_granule",
                        measurements=["nbar_red", "nbar_green", "nbar_blue"],
                        time=("2018-01-01", "2018-01-07"),
                        x=(356895, 333715),
                        y=(5729345, 5744446),
                        output_crs="EPSG:28355",
                        crs="EPSG:28355",
                        resolution=(-250, 250))

print(ds_custom_crs)

<xarray.Dataset>
Dimensions:      (time: 1, x: 94, y: 61)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-03T00:11:01.026000
  * y            (y) float64 5.744e+06 5.744e+06 ... 5.73e+06 5.729e+06
  * x            (x) float64 3.336e+05 3.339e+05 ... 3.566e+05 3.569e+05
    spatial_ref  int32 28355
Data variables:
    nbar_red     (time, y, x) int16 388 433 495 514 542 ... 299 199 318 261 295
    nbar_green   (time, y, x) int16 538 550 677 687 674 ... 407 300 414 407 400
    nbar_blue    (time, y, x) int16 637 608 725 743 746 ... 486 367 445 479 499
Attributes:
    crs:           EPSG:28355
    grid_mapping:  spatial_ref


Note that the `crs` attribute in the **Attributes** section has changed to `EPSG:28355`. 

### Spatial resampling methods
When a product is re-projected to a different CRS and/or resolution, the new pixel grid may differ from the original input pixels by size, number and alignment.
It is therefore necessary to apply a spatial "resampling" rule that allocates input pixel values into the new pixel grid.

By default, `dc.load()` resamples pixel values using "nearest neighbour" resampling, which allocates each new pixel with the value of the closest input pixel.
Depending on the type of data and the analysis being run, this may not be the most appropriate choice (e.g. for continuous data).

The `resampling` parameter in `dc.load()` allows you to choose a custom resampling method from the following options: 

```
"nearest", "cubic", "bilinear", "cubic_spline", "lanczos", 
"average", "mode", "gauss", "max", "min", "med", "q1", "q3"
```

The example cell below requests that all loaded data is resampled using "average" resampling:

In [7]:
# Note the additional `resampling` parameter
ds_averageresampling = dc.load(product="ga_s2a_ard_nbar_granule",
                               measurements=["nbar_red", "nbar_green", "nbar_blue"],
                               time=("2018-01-01", "2018-01-07"),
                               x=(356895, 333715),
                               y=(5729345, 5744446),
                               output_crs="EPSG:28355",
                               crs="EPSG:28355",
                               resolution=(-250, 250))

print(ds_averageresampling)

<xarray.Dataset>
Dimensions:      (time: 1, x: 94, y: 61)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-03T00:11:01.026000
  * y            (y) float64 5.744e+06 5.744e+06 ... 5.73e+06 5.729e+06
  * x            (x) float64 3.336e+05 3.339e+05 ... 3.566e+05 3.569e+05
    spatial_ref  int32 28355
Data variables:
    nbar_red     (time, y, x) int16 388 433 495 514 542 ... 299 199 318 261 295
    nbar_green   (time, y, x) int16 538 550 677 687 674 ... 407 300 414 407 400
    nbar_blue    (time, y, x) int16 637 608 725 743 746 ... 486 367 445 479 499
Attributes:
    crs:           EPSG:28355
    grid_mapping:  spatial_ref


Python dictionaries can be used to request different sampling methods for different measurements. 
This can be particularly useful when some measurements contain contain categorical data which require resampling methods such as "nearest" or "mode" that do not modify the input pixel values.

The example cell below specifies `resampling={"nbar_red": "nearest", "*": "average"}`, which implements "nearest" neighbour resampling for the `nbar_red` satellite band only. `"*": "average"` will apply "average" resampling for all other satellite bands:


In [8]:
ds_customresampling = dc.load(product="ga_s2a_ard_nbar_granule",
                              measurements=["nbar_red", "nbar_green", "nbar_blue"],
                              time=("2018-01-01", "2018-01-07"),
                              x=(356895, 333715),
                              y=(5729345, 5744446),
                              output_crs="EPSG:28355",
                              crs="EPSG:28355",
                              resolution=(-250, 250),
                              resampling={"nbar_red": "nearest", "*": "average"})

print(ds_customresampling)

<xarray.Dataset>
Dimensions:      (time: 1, x: 94, y: 61)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-03T00:11:01.026000
  * y            (y) float64 5.744e+06 5.744e+06 ... 5.73e+06 5.729e+06
  * x            (x) float64 3.336e+05 3.339e+05 ... 3.566e+05 3.569e+05
    spatial_ref  int32 28355
Data variables:
    nbar_red     (time, y, x) int16 388 433 495 514 542 ... 299 199 318 261 295
    nbar_green   (time, y, x) int16 566 600 655 680 666 ... 362 362 364 371 378
    nbar_blue    (time, y, x) int16 647 673 738 765 756 ... 425 432 437 448 464
Attributes:
    crs:           EPSG:28355
    grid_mapping:  spatial_ref


> For more information about spatial resampling methods, see the [following guide](https://rasterio.readthedocs.io/en/stable/topics/resampling.html)

## Loading data using the query dictionary syntax
It is often useful to re-use a set of query parameters to load data from multiple products.
To achieve this, load data using the "query dictionary" syntax.
This involves placing the query parameters inside a Python dictionary object which can be re-used for multiple data loads:

In [9]:
query = {"x": (145.1, 145.2),
         "y": (-38.4, -38.5),
         "time": ("2018-01-01", "2018-01-07"),
         "crs": "epsg:4326",
         "output_crs": "epsg:4326",
         "resolution": (-0.01, 0.01),
         "measurements": ["nbar_red", "nbar_green", "nbar_blue"]}

The query dictionary object can be added as an input to `dc.load()`. 

> The `**` syntax below is Python's "keyword argument unpacking" operator.
This operator takes the named query parameters listed in the query dictionary (e.g. `"x": (145.1, 145.2)`), and "unpacks" them into the `dc.load()` function as new arguments. 
For more information about unpacking operators, refer to the [Python documentation](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists)

In [10]:
ds = dc.load(product="ga_s2a_ard_nbar_granule",
             **query)

print(ds)

<xarray.Dataset>
Dimensions:      (latitude: 10, longitude: 10, time: 1)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-03T00:11:01.026000
  * latitude     (latitude) float64 -38.41 -38.41 -38.43 ... -38.48 -38.48 -38.5
  * longitude    (longitude) float64 145.1 145.1 145.1 ... 145.2 145.2 145.2
    spatial_ref  int32 4326
Data variables:
    nbar_red     (time, latitude, longitude) int16 714 889 788 ... 1065 1274
    nbar_green   (time, latitude, longitude) int16 878 1089 1033 ... 909 786 957
    nbar_blue    (time, latitude, longitude) int16 831 1070 1075 ... 641 573 728
Attributes:
    crs:           epsg:4326
    grid_mapping:  spatial_ref


After specifying the reusable query, it can be easily used to load data from a different product.
The example cell below loads Sentinel-2B data for the same extent, time, output CRS and resolution as the previously loaded Sentinel-2A data:

In [11]:
ds_s2b = dc.load(product="ga_s2b_ard_nbar_granule",
                 **query)

print(ds_s2b)

<xarray.Dataset>
Dimensions:      (latitude: 10, longitude: 10, time: 1)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-01T00:20:59.027000
  * latitude     (latitude) float64 -38.41 -38.41 -38.43 ... -38.48 -38.48 -38.5
  * longitude    (longitude) float64 145.1 145.1 145.1 ... 145.2 145.2 145.2
    spatial_ref  int32 4326
Data variables:
    nbar_red     (time, latitude, longitude) int16 4814 4943 517 ... 2235 2105
    nbar_green   (time, latitude, longitude) int16 4701 4799 727 ... 2237 2122
    nbar_blue    (time, latitude, longitude) int16 4576 4630 774 ... 2197 2094
Attributes:
    crs:           epsg:4326
    grid_mapping:  spatial_ref


## Other helpful tricks
### Adding a progress bar
When loading large amounts of data, it can be useful to view the progress of the data load. 
The `progress_cbk` parameter in `dc.load()` adds a progress bar that indicates how the load is progressing:

![Progress bar](../Supplementary_data/04_Loading_data/progress_bar.jpg)

The example cell below loads a full month of data (December 2017) from the `ga_s2a_ard_nbar_granule` product with a progress bar:

In [12]:
query = {"x": (356895, 333715),
         "y": (5729345, 5744446),
         "time": ("2017-12"),
         "crs": "epsg:28355",
         "output_crs": "epsg:28355",
         "resolution": (-250, 250)}

ds_progress = dc.load(product="ga_s2a_ard_nbar_granule",
                      measurements=["nbar_red", "nbar_green", "nbar_blue"],
                      progress_cbk=with_ui_cbk(),
                      **query)

print(ds_progress)

<xarray.Dataset>
Dimensions:      (time: 3, x: 94, y: 61)
Coordinates:
  * time         (time) datetime64[ns] 2017-12-07T00:20:51.026000 ... 2017-12-27T00:20:51.026000
  * y            (y) float64 5.744e+06 5.744e+06 ... 5.73e+06 5.729e+06
  * x            (x) float64 3.336e+05 3.339e+05 ... 3.566e+05 3.569e+05
    spatial_ref  int32 28355
Data variables:
    nbar_red     (time, y, x) int16 4558 4028 3448 2712 655 ... 542 472 596 460
    nbar_green   (time, y, x) int16 4464 4011 3365 2872 963 ... 605 580 674 547
    nbar_blue    (time, y, x) int16 4350 4049 3192 3060 1147 ... 676 685 715 654
Attributes:
    crs:           epsg:28355
    grid_mapping:  spatial_ref


## Recommended next steps

To continue working through the notebooks in this beginner's guide, the following notebooks are designed to be worked through in the following order:

1. [Jupyter Notebooks](01_Jupyter_notebooks.ipynb)
2. [Digital Earth Australia](02_DEA.ipynb)
3. [Products and measurements](03_Products_and_measurements.ipynb)
4. **Loading data (this notebook)**
5. [Plotting](05_Plotting.ipynb)
6. [Performing a basic analysis](06_Basic_analysis.ipynb)
7. [Introduction to Numpy](07_Intro_to_numpy.ipynb)
8. [Introduction to Xarray](08_Intro_to_xarray.ipynb)
9. [Parallel processing with Dask](09_Parallel_processing_with_dask.ipynb)

***
## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please review the FAQ section and support options on the [EY Data Science platform](https://datascience.ey.com/).